# Deep Q-learning using backprop on small grdiworld

In [1]:
import os
os.chdir("..")
from src.gym_kalman.env_Gridworld import GridworldEnv

In [2]:
# Initialize value function
import numpy as np

# initialize the environment
grid_size = 4
env = GridworldEnv(grid_size=grid_size, reward_std=0.)
num_states = env.observation_space.n
actions = np.arange(env.action_space.n)

In [3]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [6]:
def state_encoder(state):
    # one hot encoding
    state_onehot = np.zeros(num_states)
    state_onehot[state] = 1
    return state_onehot

def state_decoder(state_onehot):
    return np.argmax(state_onehot)

In [7]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 1
EPS_START = 0.9
EPS_END = 0.0001
EPS_DECAY = 1000

TAU = 0.005
LR = 1e-2

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = 16

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

def select_greedy_action(state):
    with torch.no_grad():
        return policy_net(state).max(1).indices.view(1, 1)

In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [9]:
# Extract current policy
def extract_policy(num_states, episode_i):
    policy = np.zeros(num_states)
    for state in range(num_states):
        if state == 15:  # Terminal state
            policy[state] = 10
            continue
        state_encoded = state_encoder(state)
        state_tensor = torch.tensor(state_encoded, dtype=torch.float32, device=device).unsqueeze(0)
        suggested_action = select_greedy_action(state_tensor)
        policy[state] = suggested_action

    policy_grid = np.array(policy).reshape((grid_size, grid_size))
    # Print title of the plot
    print(f"Episode {episode_i}'s policy")
    print(policy_grid)
    return

def print_state_values(num_states, i_episode):
    ########################## Print the state values ##########################
    print(f"\nEpisode {i_episode}'s state values")
    values = np.zeros(num_states)
    for state in range(num_states):
        if state == 15:  # Terminal state
            continue
        state_encoded = state_encoder(state)
        state_tensor = torch.tensor(state_encoded, dtype=torch.float32, device=device).unsqueeze(0)
        suggested_action = select_greedy_action(state_tensor)
        with torch.no_grad():
            values[state] = policy_net(state_tensor)[0].tolist()[suggested_action]

    value_grid = np.array(values).reshape((grid_size, grid_size))
    value_grid = np.round(value_grid, 2)
    print(value_grid)

In [10]:
num_episodes = 300

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = state_encoder(state)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        observation = state_encoder(observation)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            break

    print_state_values(num_states, i_episode)

# Extract policy
extract_policy(num_states, i_episode)

print('Complete')


Episode 0's state values
[[0.09 0.1  0.09 0.1 ]
 [0.11 0.17 0.11 0.12]
 [0.1  0.07 0.13 0.11]
 [0.12 0.09 0.12 0.  ]]

Episode 1's state values
[[0.09 0.1  0.09 0.1 ]
 [0.11 0.17 0.11 0.12]
 [0.1  0.07 0.13 0.11]
 [0.12 0.09 0.12 0.  ]]

Episode 2's state values
[[-0.92 -0.89 -0.94 -0.92]
 [-0.91 -0.82 -0.89 -0.91]
 [-0.93 -0.87 -0.92 -0.88]
 [-0.94 -0.92 -0.96  0.  ]]

Episode 3's state values
[[-1.23 -1.19 -1.22 -1.19]
 [-1.2  -1.14 -1.19 -1.19]
 [-1.22 -1.19 -1.19 -1.09]
 [-1.2  -1.17 -1.15  0.  ]]

Episode 4's state values
[[-1.33 -1.29 -1.32 -1.29]
 [-1.3  -1.27 -1.3  -1.29]
 [-1.32 -1.3  -1.27 -1.02]
 [-1.29 -1.26 -1.01  0.  ]]

Episode 5's state values
[[-1.39 -1.37 -1.38 -1.36]
 [-1.37 -1.35 -1.35 -1.35]
 [-1.38 -1.37 -1.33 -1.01]
 [-1.36 -1.31 -0.98  0.  ]]

Episode 6's state values
[[-1.43 -1.41 -1.43 -1.4 ]
 [-1.42 -1.4  -1.41 -1.41]
 [-1.43 -1.42 -1.37 -1.02]
 [-1.42 -1.37 -0.97  0.  ]]

Episode 7's state values
[[-1.71 -1.69 -1.7  -1.68]
 [-1.71 -1.71 -1.69 -1.62]
 [-1.72